### Using Keras with TensorFlow backend

In [1]:
import numpy as np

from keras.models import Model
from keras.layers import Flatten, Dense, Input, Activation
from keras.layers import BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D
from keras.preprocessing import image
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.models import Sequential
import h5py

Using TensorFlow backend.


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

### Set GPU
tf.device('/gpu:0') tells TF to use gpu 0. Can use gpu 1. If cell is not run then fills both GPUs

In [3]:
import tensorflow as tf
tf.device('/gpu:0')

In [2]:
#val = np.loadtxt('kaggle/val.txt',dtype='str',delimiter=' ')

In [3]:
#Grab image paths put in list
#path = 'kaggle/train_sm/'
#img_paths = []
#for i in val:
#    img_paths.append(path + i[0])
#print len(img_paths)

688


### Grab Test Image file paths and put into List

In [3]:
import os

In [4]:
#Grab image paths put in list
path = 'UCMerced/Images/'
img_paths = []
label = []
for i in sorted(os.listdir(path)):
    for j in sorted(os.listdir(path+i)):
        img_paths.append(path + i + '/'+ j)
        label.append(j)
print len(img_paths)

2100


In [4]:
new_path = 'Cifar/Val/'
#Grab image paths put in list
img_paths = []
for i in sorted(os.listdir(new_path)):
    small_path = new_path+i+'/'
    for j in sorted(os.listdir(small_path)):
        img_paths.append(small_path + j)
print len(img_paths)

10000


In [58]:
#val = np.loadtxt('kaggle/val.txt',dtype='str',delimiter=' ')

In [64]:
#Grab Training image paths put in list
#path = 'kaggle/train_sm/'
#img_paths = []
#labels = []
#for i in val:
#    img_paths.append(path + i[0])
#    labels.append(i[1])
#print len(img_paths)

688


### Import Functional Keras model from keras.applications Library 

In [5]:
#from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16

### Create Instance of base model 
include_top = False gets rid of top model.

weights = 'imagenet' Sets weights to image net weights.

In [6]:
#weights_path = 'Caffe models/ResNet50/weights/ResNet50-finetune4-59-0.83.h5'
#model = ResNet50(include_top=False, weights='imagenet', input_tensor=Input(shape=(224,224,3)))

model = VGG16(include_top=False, weights='imagenet', input_tensor=Input(shape=(224,224,3)))


### Build Top Model

In [7]:
# ResNet Top Model
x = model.output
x = Flatten(name='flatten')(x) 
x = BatchNormalization(axis=1, name='batch_norm')(x)

# 344 is the number of classes
x = Dense(344, name='fc1000')(x)
x = Activation("softmax",name='bfc1000')(x)


### Create a New Model Class where input is base model and output is Top model

In [8]:
# Stack top_model on top
final_model = Model(input=model.input, output=x)

In [9]:
#File handle to open weights path
weights_path = 'Caffe models/VGG/VGG16-finetune2-24-0.68.h5'
f = h5py.File(weights_path)

In [14]:
#K.get_session().run(tf.global_variables_initializer())

### For each layer set the weights from the .h5 file

In [10]:
for k in range(len(f.attrs['layer_names'])): 
    name = f.attrs['layer_names'][k]
    weights = [f[name][h][()] for h in f[name].keys()]
    print k
    final_model.layers[k].set_weights(weights)
f.close()
print('Model loaded.')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
Model loaded.


### Create a new model to feature extract and any desired layer

In [11]:
layer_name = 'fc1000'
intermediate_layer_model = Model(input=final_model.input,
                                 output=final_model.get_layer(layer_name).output)
#intermediate_layer_model = Model(input=model.input,
#                                 output=model.get_layer('avg_pool').output)

In [12]:
def preprocess_input(x):
    x[:, :, :, 0] -= 103.939
    x[:, :, :, 1] -= 116.779
    x[:, :, :, 2] -= 123.68
    # 'RGB'->'BGR'
    x = x[:, :, :, ::-1]
    x = x*(1./255)
    return x

In [13]:
#Test for 1 image 
img = image.load_img(img_paths[0], target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
test = intermediate_layer_model.predict(x)

In [ ]:
test

In [23]:
#Get the first weight of each layer to check how big by sending image through
for name in f.attrs['layer_names']:
    try:
        model.get_layer(name).output
    except:
        continue
    intermediate_layer_model = Model(input=model.input,
                                     output=model.get_layer(name).output)
    img = image.load_img(img_paths[0], target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    test = intermediate_layer_model.predict(x)
    print name
    print test[0].flatten()[0]


input_1
-0.308549
zeropadding2d_1
0.0
conv1
-0.440767
bn_conv1
-0.183358
activation_1
0.0
maxpooling2d_1
1.85875
res2a_branch2a
-3.10517
bn2a_branch2a
0.93697
activation_2
0.93697
res2a_branch2b
0.644813
bn2a_branch2b
0.993948
activation_3
0.993948
res2a_branch2c
0.233095
res2a_branch1
0.209055
bn2a_branch2c
0.430848
bn2a_branch1
-2.06266
merge_1
-1.63181
activation_4
0.0
res2b_branch2a
1.35118
bn2b_branch2a
1.52387
activation_5
1.52387
res2b_branch2b
0.941229
bn2b_branch2b
-1.32862
activation_6
0.0
res2b_branch2c
0.145668
bn2b_branch2c
2.8737
merge_2
2.8737
activation_7
2.8737
res2c_branch2a
-4.26551
bn2c_branch2a
3.15851
activation_8
3.15851
res2c_branch2b
-0.0446791
bn2c_branch2b
1.48858
activation_9
1.48858
res2c_branch2c
-0.793427
bn2c_branch2c
2.24055
merge_3
5.11425
activation_10
5.11425
res3a_branch2a
-8.3718
bn3a_branch2a
-3.3707
activation_11
0.0
res3a_branch2b
-2.88716
bn3a_branch2b
2.74654
activation_12
2.74654
res3a_branch2c
-0.253533
res3a_branch1
-0.157375
bn3a_branch2c


In [15]:
features = []
for i in range(len(img_paths)):
    img = image.load_img(img_paths[i], target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features.append(intermediate_layer_model.predict(x).flatten())

In [16]:
image_features = np.squeeze(features)

In [17]:
image_features.shape

(2100, 344)

### Process to Save extracted Features into Dataframe then to CSV

In [18]:
import pandas as pd

In [19]:
df1 = pd.DataFrame(np.array(label))
df2 = pd.DataFrame(image_features)

In [20]:
frames = [df1, df2]
result = pd.concat(frames,axis=1)

In [21]:
result.head()

,0,0,1,2,3,4,5,6,7,8,...,334,335,336,337,338,339,340,341,342,343
0,agricultural00.tif,-17.498186,-11.243931,-2.828087,-5.019659,-4.800480,-0.314717,-1.240391,-10.934299,-13.917276,...,-19.817291,-5.006012,-12.085507,-0.838910,2.503154,-4.625121,-10.408319,-8.315767,-5.782712,-9.969353
1,agricultural01.tif,-17.414495,-10.923462,-2.705335,-4.852305,-4.538103,-0.125256,-1.075972,-10.810599,-13.663400,...,-19.632097,-4.853601,-11.882241,-0.660391,2.527540,-4.467528,-10.178945,-8.181026,-5.690276,-9.741242
2,agricultural02.tif,-17.590063,-11.227840,-2.726588,-4.989417,-4.771765,-0.278810,-1.081569,-10.944439,-14.005670,...,-19.861160,-4.985163,-12.073009,-0.787942,2.364157,-4.569351,-10.536433,-8.334929,-5.815950,-9.914953
3,agricultural03.tif,-17.511562,-11.085929,-2.736357,-4.907364,-4.601976,-0.205986,-1.070554,-10.920285,-13.756444,...,-19.591187,-4.896352,-11.999438,-0.660690,2.544987,-4.498331,-10.237048,-8.201381,-5.712914,-9.778503
4,agricultural04.tif,-18.363272,-11.981666,-2.868964,-5.673901,-5.192054,-0.712970,-1.946154,-11.435550,-14.883910,...,-20.801493,-5.596359,-12.954803,-1.534384,2.228042,-5.711013,-11.136153,-8.987082,-6.150424,-10.468662


In [22]:
result.to_csv('Features Extracted/UCMerced/Finetune/Single/UCM-VGG16-24epoch-fc1000-68-fine2.csv',index=False)